# 1. Importation Des Librairies 

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# pour le spliting
from sklearn.model_selection import train_test_split 
# pour la standardisationdes data 
from sklearn.preprocessing import StandardScaler
#importation pour le snn
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import warnings 
warnings.filterwarnings('ignore')

# 2. Importation De La Data Set

In [93]:
data = pd.read_csv('data/Darknet_2.csv')
data.head(2)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,7/24/2015 16:09,229,1,1,...,0,0,0,0,0.0,0.0,0.0,0.0,Non-Tor,AUDIO-STREAMING
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,7/24/2015 16:09,407,1,1,...,0,0,0,0,0.0,0.0,0.0,0.0,Non-Tor,AUDIO-STREAMING


# 3. Analyse Des Data

In [94]:
#data.info()

In [95]:
data.isnull().sum()

Flow ID     0
Src IP      0
Src Port    0
Dst IP      0
Dst Port    0
           ..
Idle Std    0
Idle Max    0
Idle Min    0
Label       0
Label.1     0
Length: 85, dtype: int64

In [96]:
data.shape

(141530, 85)

In [97]:
data['Label.1'].unique()

array(['AUDIO-STREAMING', 'Browsing', 'Chat', 'Email', 'File-Transfer',
       'File-transfer', 'P2P', 'Video-Streaming', 'Audio-Streaming',
       'Video-streaming', 'VOIP'], dtype=object)

In [98]:
data['Label'].unique()

array(['Non-Tor', 'NonVPN', 'Tor', 'VPN'], dtype=object)

# 4. Traitement Des Data 

In [99]:
# reduction à deux categories

In [100]:
data['labelfinal'] = data['Label'].apply(lambda x: 1 if x in ['Tor','VPN'] else 0)
data['labelfinal'].unique()

array([0, 1])

In [101]:
data.head(1)

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1,labelfinal
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,7/24/2015 16:09,229,1,1,...,0,0,0,0.0,0.0,0.0,0.0,Non-Tor,AUDIO-STREAMING,0


In [102]:
# mapping de label 1 

In [103]:
Mapping_lab1 ={
    'AUDIO-STREAMING':0,
    'Browsing': 1,
    'Chat':2,
    'Email':3,
    'File-Transfer' :4,
    'File-transfer' : 5,
    'P2P' : 6,
    'Video-Streaming' : 7,
    'Audio-Streaming':8,
    'Video-streaming':9,
     'VOIP':10
}
data['Label.1']= data['Label.1'].map(Mapping_lab1)
data['Label.1'].unique()


    

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [104]:
# suppression des colonnes inutiles 
#data.info()

In [105]:
data.drop(columns = ['Flow ID', 'Src IP', 'Dst IP', 'Timestamp', 'Label'], inplace = True, axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141530 entries, 0 to 141529
Data columns (total 81 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Src Port                    141530 non-null  int64  
 1   Dst Port                    141530 non-null  int64  
 2   Protocol                    141530 non-null  int64  
 3   Flow Duration               141530 non-null  int64  
 4   Total Fwd Packet            141530 non-null  int64  
 5   Total Bwd packets           141530 non-null  int64  
 6   Total Length of Fwd Packet  141530 non-null  int64  
 7   Total Length of Bwd Packet  141530 non-null  int64  
 8   Fwd Packet Length Max       141530 non-null  int64  
 9   Fwd Packet Length Min       141530 non-null  int64  
 10  Fwd Packet Length Mean      141530 non-null  float64
 11  Fwd Packet Length Std       141530 non-null  float64
 12  Bwd Packet Length Max       141530 non-null  int64  
 13  Bwd Packet Len

# 5. Division Des Données 

In [106]:
X = data.iloc[ : , :-1]
y = data.iloc[ : , -1]
x_train, x_test, y_train, y_test = train_test_split(X, y , test_size = 0.2 , random_state= 40)

print ("le x_train shape est :", x_train.shape)
print ("le x_test shape est :", y_train.shape)
print ("le y_train shape est :", x_test.shape)
print ("le y_test shape est :", y_test.shape)



le x_train shape est : (113224, 80)
le x_test shape est : (113224,)
le y_train shape est : (28306, 80)
le y_test shape est : (28306,)


In [107]:
# standardisation des data
#scaler = StandardScaler()

In [108]:
#x_train = scaler.fit_transform(x_train)
#x_test = scaler.fit_transform(x_test)
x_train.shape



(113224, 80)

# 6. Construction Du SNN

In [109]:
model_snn = Sequential()

In [110]:
# ajout de la couche 

model_snn.add(Dense(64, input_shape = (80,), activation='relu'))

# ajout de la couche de sortie
# la couche de sorti prend comme dimmension le nombre de colonne 
model_snn.add(Dense(1, input_shape = (1,), activation =  'sigmoid'))

# compiler le modele 

model_snn.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model_snn.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 64)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,249 (20.50 KB)

 Trainable params: 5,249 (20.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# entrainnement du model 
history = model_snn.fit(x_train, y_train, epochs=150, verbose =1 , validation_data = (x_test, y_test))

Epoch 1/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8117 - loss: 3573921611776.0000 - val_accuracy: 0.8279 - val_loss: 0.4595
Epoch 2/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8281 - loss: 0.4590 - val_accuracy: 0.8279 - val_loss: 0.4592
Epoch 3/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8273 - loss: 0.4601 - val_accuracy: 0.8279 - val_loss: 0.4592
Epoch 4/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8276 - loss: 0.4597 - val_accuracy: 0.8279 - val_loss: 0.4592
Epoch 5/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.8270 - loss: 0.4607 - val_accuracy: 0.8279 - val_loss: 0.4592
Epoch 6/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8289 - loss: 0.4576 - val_accuracy: 0.8279 - val_loss: 0.4592
Epoch 7/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.8273 - loss: 0.4601 - val_accuracy: 0.8279 - val_loss: 0.4592
Epoch 8/150
3539/3539 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.82

# 7. Avaluation Du Model 

In [ ]:
epoch_accuracy = history.history['accuracy']
epoch_loss = history.history['loss']

epoch_val_acc = history.history['val_accuracy']
epoch_val_loss = history.history['val_loss']


In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(range(0, len(epoch_accuracy)), epoch_accuracy, color='blue', label='Training Accuracy')
plt.plot(range(0, len(epoch_loss)), epoch_loss, color='red', label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Valeur')
plt.title('Accuracy Vs Loss')
plt.legend()
plt.plot()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(range(0, len(epoch_accuracy)), epoch_accuracy, color='blue', label='Training Accuracy')
plt.plot(range(0, len(epoch_val_acc)), epoch_val_acc, color='red', label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy Vs Test Accuracy')
plt.legend()
plt.plot()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(range(0, len(epoch_loss)), epoch_loss, color='blue', label='Training Loss')
plt.plot(range(0, len(epoch_val_loss)), epoch_val_loss, color='red', label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Vs Test Loss')
plt.legend()
plt.plot()
plt.show()

# 8. Evaluation Du Model 

In [ ]:
model_snn.evaluate(x_test, y_test)